# Portfolio Optimization with Linear and Fixed Transaction Costs

The paper "Portfolio Optimization with Linear and Fixed Transaction Costs" by Miguel Sousa Lobo, Maryam Fazel and Stephen Boyd considers the problem of portfolio optimization with variable transaction costs as a nonconvex optimization problem that can be broken down into convex optimization subproblems. Their method results in a suboptimal portfolio and an upper bound on the actual optimal portfolio, and they find that in practice the gap between the two is small.

The portfolio selection problem in the paper is formulated as such:

$
\begin{aligned}
& \underset{x}{\text{maximize}}
& & \bar{a}^T (w + x) \\
& \text{subject to}
& & 1^T x + \phi(x) \leq 0 \\
& & & w + x \in S
\end{aligned}
$

where, for a portfolio with investments in $n$ assets,

$\bar{a}$ is the vector of expected returns on each asset, <br>
$w$ is the vector of current holdings in each asset, <br>
$x$ is the vector of amounts transacted in each asset, $x_i < 0$ sell $x$ units of asset $i$, $x_i > 0$ buy <br>
$\phi(x)$ is the transaction cost function, <br>
$S$ is the set of feasible portfolios.

Evidently, it follows that the problem of minimizing the total transaction costs subject to portoflio constraints can be formulated as such:

$
\begin{aligned}
& {\text{minimize}}
& & \phi(x) \\
& \text{subject to}
& & \bar{a}^T (w + x) \geq r_{\text{min}} \\ 
& & & w + x \in S
\end{aligned}
$

where $r_{\text{min}} $ is the desired lower bound on the expected return.

The paper focuses on the original optimization problem more, however this newly constructed optimization problem is of greater use to us.

In [19]:
import cvxpy as cp
import numpy as np

In [70]:
np.random.seed(0) # comment out to try different values

n = 5  # number of assets

w = np.random.randint(1, 101, n)  # current holdings in each asset
a_bar = np.ones(n) + np.random.rand(n)  # assuming positive expected returns on each asset
r_min = 1.5  # want a 50% return

print("Current portfolio:", w)
print("Expected return on each holding:", a_bar)

Current portfolio: [45 48 65 68 68]
Expected return on each holding: [1.84725174 1.6235637  1.38438171 1.29753461 1.05671298]


Our current portfolio value and expected portfolio value can be calculated as:

In [71]:
print("Current portfolio value:", sum(w), "vs. expected after returns {:.2f}".format(a_bar.T @ w))

Current portfolio value: 294 vs. expected after returns 411.13


An initial investment of 294 turns to 411.13 after holding period. That's an almost 40% return! But we can do better.

However, we see that we own 68 shares in the 5th asset of our portfolio which only has a return of 5.6% compared to a whopping 84% return on our smallest position. Intuitively, we know there should be a rebalancing of some sorts here.

We need to define our transaction cost function first. The paper implements a linear function of the form:

$\phi(x) = \beta_i + \alpha_i|x_{\text{i}}|, $

where $\alpha_i$ are transaction costs associated with trading asset $i$ and $\beta_i$ are fixed costs. The inclusion of fixed costs discourages swapping small amounts within our portfolio. 

However, for the sake of simplicty, we will ignore including $\alpha_i$ and $\beta_i$ in our function. We will see a caveat of this simplifying assumption $(\beta_i = 0)$ in our results. 

In [75]:
# Transaction cost function
def phi(x):
    return cp.sum(cp.abs(x))


x = cp.Variable(n) # variable that can be adjusted

objective = cp.Minimize(phi(x)) # objective function

constraints = [
    a_bar.T @ (w + x) / sum(w) >= r_min,  # minimum expected return
    cp.sum(w + x) == cp.sum(w)  # total value in portfolio must remain same
]

problem = cp.Problem(objective, constraints)
result = problem.solve()

print("Optimal value of transaction cost:", problem.value)
print("Optimal transactions x:", x.value)

Optimal value of transaction cost: 6.114991448975474e-09
Optimal transactions x: [ 2.69757050e-09  1.69897152e-09  4.42780753e-10  9.30081692e-12
 -1.26636785e-09]


In [76]:
print("Optimized portfolio:", w + x.value)

Optimized portfolio: [82.78305261 48.00000002 64.99999999 67.99999998 30.2169474 ]


Notice the purchase of an additional .00000002 shares in our 2nd asset. While this is the optimal portfolio given our constraints, we know this will rarely be profitable in a real world scenario. However, our simple model allows us formulate a function for implicit transaction costs and minimize them.

### References

Lobo, M.S., Fazel, M. & Boyd, S. Portfolio optimization with linear and fixed transaction costs. Ann Oper Res 152, 341–365 (2007).<br>
https://doi.org/10.1007/s10479-006-0145-1